#### This is a testing notebook for creating a database and checking out features

In [1]:
import sqlite3
import os
import json

# Function to create database and tables
def create_database():
    conn = sqlite3.connect('product_database10.db')
    c = conn.cursor()

    # Create products table
    c.execute('''CREATE TABLE IF NOT EXISTS gaming_laptops (
                    id INTEGER PRIMARY KEY,
                    url TEXT,
                    title TEXT,
                    warranty INTEGER,
                    regular_price REAL,
                    happy_price REAL,
                    category TEXT
                 )''')

    # Create feature_products table
    c.execute('''CREATE TABLE IF NOT EXISTS feature_gaming_laptops (
                    id INTEGER PRIMARY KEY,
                    feature_name TEXT,
                    feature_value TEXT
                 )''')

    # Create product_feature table
    c.execute('''CREATE TABLE IF NOT EXISTS gaming_laptop_feature (
                    product_id INTEGER,
                    feature_id INTEGER,
                    FOREIGN KEY (product_id) REFERENCES gaming_laptops(id),
                    FOREIGN KEY (feature_id) REFERENCES feature_gaming_laptops(id)
                 )''')

    conn.commit()
    conn.close()

# Function to insert product data into database
def insert_product_data(product_data):
    conn = sqlite3.connect('product_database10.db')
    c = conn.cursor()

    # Insert product data into products table
    c.execute('''INSERT INTO gaming_laptops (url, title, warranty, regular_price, happy_price, category)
                 VALUES (?, ?, ?, ?, ?, ?)''', (
                    product_data.get('url'),
                    product_data.get('title'),
                    product_data.get('warranty'),
                    product_data.get('regular_price'),
                    product_data.get('happy_price'),
                    product_data.get('category')
                 ))
    product_id = c.lastrowid

    # Insert feature data into feature_gaming_laptops table and product_feature table
    for feature, value in product_data.get('features', {}).items():
        # Check if feature already exists
        c.execute('''SELECT id FROM feature_gaming_laptops WHERE feature_name = ? AND feature_value = ?''', (feature, value))
        feature_row = c.fetchone()
        if feature_row:
            feature_id = feature_row[0]
        else:
            c.execute('''INSERT INTO feature_gaming_laptops (feature_name, feature_value) VALUES (?, ?)''', (feature, value))
            feature_id = c.lastrowid

        # Insert into product_feature table
        c.execute('''INSERT INTO gaming_laptop_feature (product_id, feature_id) VALUES (?, ?)''',
                  (product_id, feature_id))

    conn.commit()
    conn.close()

# Function to read JSON files from a directory and insert data into database
def process_json_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r') as file:
                product_data = json.load(file)
                insert_product_data(product_data)

# Create database and tables
create_database()

# Process JSON files and insert data into database
json_directory = r'C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\gaming_laptops\gaming_laptops_flattened'
process_json_files(json_directory)


In [2]:
def get_feature_values():
    conn = sqlite3.connect('product_database10.db')
    c = conn.cursor()

    # Query to get the feature names and their corresponding values
    c.execute('''SELECT feature_name, GROUP_CONCAT(feature_value) 
                 FROM feature_gaming_laptops 
                 GROUP BY feature_name''')
    
    feature_values = c.fetchall()
    conn.close()

    for feature_name, values in feature_values:
        value_list = values.split(',')
        unique_values = list(set(value_list))  # Get unique values
        print(f"For {feature_name} there are: {', '.join(unique_values)}")

# Call the function to print feature values
get_feature_values()

For 3.5 mm Combo Audio Jack port there are: 1 x 3.5mm Combo Audio Jack
For Adaptive-Sync Technology there are: G-Sync
For Audio there are:  Realtek® ALC3287 codec, Hi-Res certification
AI noise-canceling technology
Dolby Atmos
Hi-Res certification
Smart Amp Technology
Security BIOS Administrator Password and User Password Protection,  Dual speakers, Тех AI noise-canceling - Dolby Atmos, Audio by B&O Dual speakers HP Audio Boost,  Hi-Res certification, Tech AI noise-canceling
Dolby Atmos
Hi-Res certification
AI noise-canceling technology
Dolby Atmos
Smart Amp Technology,  HP Audio Boost, 2x 2W speakers, Dolby Atmos, Tech AI noise-canceling - Dolby Atmos,  Realtek ALC3287 codec,  Realtek® ALC3306 codec, High Definition (HD) Audio, Dolby Atmos
Hi-Res certification
Smart Amp Technology
Support Microsoft Cortana near field/Far field, DTS:X® Ultra Audio Technology, Hi-Res certification
AI noise-canceling technology
Dolby Atmos
Smart Amp Technology, 2-speaker system
AI noise-canceling technol